In [1]:
import nncf

INFO:nncf:NNCF initialized successfully. Supported frameworks detected: torch, onnx, openvino


In [2]:
!ls

best_openvino_model			   deepsparse_prune_quant.ipynb
best_opset13.onnx			   detect4
best_opset19.onnx			   graph.xml
best_prune_75_quantized_int4_opset13.onnx  grath.json
best_prune_75_quantized_int4_opset19.onnx  lab3.ipynb
best_prune_75_quantized_int8_opset13.onnx  lab3_py3.11.ipynb
best_prune_75_quantized_int8_opset19.onnx  micro.yaml
best_prune_openvino_model		   onnx_quant_prune.ipynb
best_prune_opset13.onnx			   onnx_quant_prune-opset19.ipynb
best_prune_opset19.onnx			   openvino.ipynb
best_prune.pt				   prune_.ipynb
best.pt					   runs


In [3]:
from ultralytics import YOLO

model = YOLO("best_prune_openvino_model")  

WARNING ⚠️ Unable to automatically guess model task, assuming 'task=detect'. Explicitly define task for your model, i.e. 'task=detect', 'segment', 'classify','pose' or 'obb'.


In [4]:
!ls best_prune_openvino_model

best_prune.bin	best_prune.xml	metadata.yaml


In [5]:
import openvino as ov

core = ov.Core()

det_ov_model = core.read_model('best_prune_openvino_model/best_prune.xml')

In [6]:
!ls ../test.yaml

../test.yaml


In [7]:


from ultralytics.utils import DEFAULT_CFG
from ultralytics.cfg import get_cfg
from ultralytics.data.converter import coco80_to_coco91_class
from ultralytics.data.utils import check_det_dataset

args = get_cfg(cfg=DEFAULT_CFG)
args.data = str('../test.yaml')



In [8]:
!ls ../annotations_s

images	labels


In [9]:
det_validator = model.task_map[model.task]["validator"](args=args)

In [10]:
micro = '''
train: /home/void/ml_oleg/annotations_micro/images/train  # Path to training images
val: /home/void/ml_oleg/annotations_micro/images/val      # Path to validation images

labels:
  train: /home/void/ml_oleg/annotations_micro/labels/train  # Path to training labels
  val: /home/void/ml_oleg/annotations_micro/labels/val      # Path to validation labels


names:
  0: Long snouted lancetfish
  1: Roudie scolar
  2: Marlin
  3: Swordfish
  4: Great barracuda
  5: Thresher shark
  6: Lancetfish
  7: Pomfret
  8: Tuna
  9: Opah
  10: Pelagic stingray
  11: Mahi mahi
  12: Striped marlin
  13: Wahoo
  14: No fish
  15: Human
  16: Skipjack tuna
  17: Yellowfin tuna
  18: Unknown
  19: Blue marlin
  20: Oilfish
  21: Bigeye tuna
  22: Snake mackerel
  23: Indo Pacific sailfish
  24: Water
  25: Sickle pomfret
  26: Albacore
  27: Brama
  28: Black marlin
  29: Shortbill spearfish
  30: Mola mola
  31: Rainbow runner
  32: Escolar
  33: Shark'''
with open('micro.yaml','w') as f:
    f.write(micro)

In [11]:
import yaml
from ultralytics.data import YOLODataset
with open('micro.yaml', 'r') as file:
    data = yaml.safe_load(file)
ds = YOLODataset(data=data,img_path='../annotations_micro')

Scanning ../annotations_micro/labels/train... 2 images, 0 backgrounds, 0 corrupt: 100%|██████████| 2/2 [00:00<00:00, 591.58it/s]

New cache created: ../annotations_micro/labels/train.cache


In [12]:
from ultralytics.data import build_dataloader
det_data_loader = build_dataloader(ds,batch=1,workers=2)


In [13]:
import nncf
from typing import Dict


def transform_fn(data_item:Dict):
    
    input_tensor = det_validator.preprocess(data_item)['img'].numpy()
    return input_tensor


quantization_dataset = nncf.Dataset(det_data_loader, transform_fn)

In [14]:
from nncf.common.graph import NNCFGraph

In [15]:
type(det_ov_model)

openvino.runtime.ie_api.Model

In [16]:
# from nncf.common.factory import NNCFGraphFactory

In [17]:
# NNCFGraphFactory.create(det_ov_model)

In [18]:
# from nncf.experimental.common.graph.netron import save_for_netron

# save_for_netron(NNCFGraphFactory.create(det_ov_model), "graph.xml")

In [19]:
import json
with open('grath.json') as f:
    data_graph = json.load(f)


In [20]:
names = list(map(lambda x: x['@name'],data_graph['net']['layers']['layer']))

In [21]:
out = list(filter(lambda x: '22.dfl' in x , names))

In [22]:
in_d = list(set(filter(lambda x: '__module.model.22' in x , names))-set(out))

In [23]:
# ['__module.model.22/aten::cat/Concat',
#                               '__module.model.22/aten::cat/Concat_1',
#                               '__module.model.22/aten::cat/Concat_2']

In [24]:
nncf.QuantizationPreset

<enum 'QuantizationPreset'>

In [25]:
[e.value for e in nncf.QuantizationPreset]

['performance', 'mixed']

In [26]:
# quantized_model = nncf.quantize(model, calibration_dataset)

In [27]:
ignored_scope = nncf.IgnoredScope( # post-processing
    subgraphs=[
        nncf.Subgraph(inputs=in_d,
                      outputs=out)
    ]
)

# Detection model
quantized_det_model = nncf.quantize(
    det_ov_model,
    quantization_dataset,
    preset=nncf.QuantizationPreset.MIXED,
    ignored_scope=ignored_scope
)

In [ ]:
# ааа в один поток считаеться

In [28]:
ov.save_model(quantized_det_model, 'openvino_prune_quant')